In [ ]:
!pip install -r ./requirements/pypi.txt

In [ ]:
from train import *
import torch.nn.functional as F
from torchsummary import summary
import PIL.Image
from timeit import default_timer as timer

In [ ]:
trainL, testL, valL = prepare_dataloader()

In [ ]:
images, labels = next(iter(trainL))

---

To compile model, run the below cell:

In [ ]:
model = init_model()

In [ ]:
summary(model, tuple(images.shape[1:]))

In [ ]:
compiled = torch.compile(model, mode="max-autotune", fullgraph=True)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(10):
    compiled = training_loop(compiled, optimizer, criterion, trainL, epoch)

In [ ]:
torch.save(compiled.state_dict(), "pneunomia-cnn.compiled.pth")

---

Training model loop:

In [ ]:
model = init_model()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

---
Save the model to BentoML model store:

In [ ]:
bentomodel = bentoml.pytorch.save_model("pneumonia-cnn", model)

---
Load back runners and run locally:

In [ ]:
runner = bentoml.pytorch.get("resnet-pneumonia").to_runner()
runner.init_local()
res = runner.run(images)

In [ ]:
accuracy(res, labels)

In [ ]:
acc, preds = accuracy(res, labels)
acc.item()

In [ ]:
preds[0].item(), trainL.dataset.classes

In [ ]:
trainL.dataset.classes[preds[0].item()]

In [ ]:
tests = "./data/chest_xray/chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg"
im = PIL.Image.open(tests)
im

In [ ]:
preprocess = T.Compose(
    [
        T.Resize(size=256),
        T.CenterCrop(size=224),
        T.Grayscale(num_output_channels=3),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)
preprocess(im).shape

---
Using PreTrained ResNet

In [ ]:
model = init_model()

In [ ]:
summary(model, tuple(images.shape[1:]))

In [ ]:
class_to_idx = trainL.dataset.class_to_idx
idx_to_class = {idx: cls for cls, idx in class_to_idx.items()}
class_to_idx, idx_to_class

In [ ]:
classifier_dict = torch.load("./resnet50-pneumonia.pt")
model.load_state_dict(classifier_dict)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [ ]:
model, history = training(
    model, criterion, optimizer, trainL, valL, "resnet50-pneumonia.pt"
)

In [ ]:
bentomodel = bentoml.pytorch.save_model(
    "resnet-pneumonia",
    model,
    metadata={"idx2cls": idx_to_class, "cls2idx": class_to_idx},
)

In [ ]:
runner = bentoml.pytorch.get("resnet-pneumonia").to_runner()
runner.init_local()

In [ ]:
res = runner.run(preprocess(im).unsqueeze(0))
_, pred = torch.max(res, 1)
pred.item()

In [ ]:
bentomodel.info.metadata

In [ ]:
res